In [36]:
using Pkg; Pkg.activate("C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods")
using Revise
using OrbitalNeuralMethods
import OrbitalNeuralMethods as onm

  Activating project at `C:\Users\KarlH\Desktop\ComputationalPhysicsMaster\Code\OrbitalNeuralMethods`


In [37]:
import ForwardDiff as fd
import ReverseDiff as rd
import LinearAlgebra as la
using BenchmarkTools: @btime

In [51]:
n = 1000
x = rand(n);

In [52]:
nn = NeuralNetwork([Dense(32), Sigmoid, Dense(32), Sigmoid, Dense(1)], n);
nn.old_output[1]= onm.model!(nn, x)[1];

In [53]:
@btime onm.kinetic(x, nn)

  10.614 ms (1 allocation: 16 bytes)


1.4156168160679784e-8

### Reverse

In [41]:
result = zeros(n, n)
config = rd.HessianConfig(x)
tape = rd.compile(rd.HessianTape(x -> onm.diffable_model(nn.layers, x)[1], x, config))

typename(ReverseDiff.CompiledTape)(#17)

In [42]:
function reverse(x, tape, result, nn)
    rd.hessian!(result, tape, x)
    return -0.5 * la.tr(result) / nn.old_output[1]
end

reverse (generic function with 1 method)

In [43]:
@btime reverse(x, tape, result, nn)

  71.267 ms (1 allocation: 16 bytes)


-5.649981802916813e-7

### Forward

In [44]:
result = zeros(n, n)
config = fd.HessianConfig(nothing, x);

In [45]:
function forward(x, config, result, nn)
    fd.hessian!(result, x -> onm.diffable_model(nn.layers, x)[1], x, config)
    return -0.5 * la.tr(result) / nn.old_output[1]
end

forward (generic function with 1 method)

In [46]:
@btime forward(x, config, result, nn)

  27.282 ms (1227 allocations: 20.50 MiB)


-5.649981802916828e-7

### Reverse over forward

In [47]:
result = zeros(n, n)
config = rd.JacobianConfig(x)
tape = rd.compile(rd.JacobianTape(x -> fd.gradient(x -> onm.diffable_model(nn.layers, x)[1], x), x, config))

typename(ReverseDiff.CompiledTape)(#21)

In [48]:
function reverseforward(x, tape, result, nn)
    rd.jacobian!(result, tape, x);
    return -0.5 * la.tr(result) / nn.old_output[1]
end

reverseforward (generic function with 1 method)

In [49]:
@btime reverseforward(x, tape, result, nn)

  1.231 s (1 allocation: 16 bytes)


-5.649981802916813e-7